In [2]:
import numpy as np
import cv2
from mss import mss
from PIL import Image
import time
import matplotlib.pyplot as plt
#import pyautogui
import pydirectinput
import keyboard

In [3]:
#bounding_box = {"top":325, "left":300, "width":370, "height":385} #tetr.io (field, hold, and next)
#bounding_box = {"top":325, "left":396, "width":168, "height":378} #tetr.io (field)
#bounding_box = {"top":325, "left":300, "width":370, "height":385} #tetr.io (hold)
#bounding_box = {"top":385, "left":580, "width":80, "height":242} #tetr.io (next)
bounding_box = {"top":325, "left":396, "width":270, "height":378} #tetr.io (field and next)

In [4]:
with mss() as sct:
    while True:
        screenshot = sct.grab(bounding_box)
        cv2.imshow("screen", np.array(screenshot))
        if (cv2.waitKey(1) & 0xFF) == ord('q'):
            cv2.destroyAllWindows()
            break

In [10]:
img = np.array(screenshot)
cv2.imshow("s", img[110:160,185:265])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
O_CONTOURS = (np.array([[[23,  8]],

       [[23, 41]],

       [[56, 41]],

       [[56,  8]]], dtype="int32"),)
J_CONTOURS = (np.array([[[15,  8]],

       [[15, 41]],

       [[64, 41]],

       [[64, 25]],

       [[31, 25]],

       [[30, 24]],

       [[30,  8]]], dtype="int32"),)
S_CONTOURS = (np.array([[[31,  8]],

       [[31, 24]],

       [[30, 25]],

       [[15, 25]],

       [[15, 41]],

       [[47, 41]],

       [[47, 25]],

       [[48, 24]],

       [[64, 24]],

       [[64,  8]]], dtype="int32"),)
Z_CONTOURS = (np.array([[[15,  8]],

       [[15, 24]],

       [[30, 24]],

       [[31, 25]],

       [[31, 41]],

       [[64, 41]],

       [[64, 25]],

       [[48, 25]],

       [[47, 24]],

       [[47,  8]]], dtype="int32"),)
L_CONTOURS = (np.array([[[48,  8]],

       [[48, 24]],

       [[47, 25]],

       [[15, 25]],

       [[15, 41]],

       [[64, 41]],

       [[64,  8]]], dtype="int32"),)
T_CONTOURS = (np.array([[[31,  8]],

       [[31, 24]],

       [[30, 25]],

       [[15, 25]],

       [[15, 41]],

       [[64, 41]],

       [[64, 25]],

       [[48, 25]],

       [[47, 24]],

       [[47,  8]]], dtype="int32"),)
I_CONTOURS = (np.array([[[ 6, 17]],

       [[ 6, 32]],

       [[73, 32]],

       [[73, 17]]], dtype="int32"),)

In [6]:
def contourToMino(contour):
    if np.array_equal(contour, S_CONTOURS):
        return "S"
    elif np.array_equal(contour, Z_CONTOURS):
        return "Z"
    elif np.array_equal(contour, J_CONTOURS):
        return "J"
    elif np.array_equal(contour, L_CONTOURS):
        return "L"
    elif np.array_equal(contour, T_CONTOURS):
        return "T"
    elif np.array_equal(contour, O_CONTOURS):
        return "O"
    elif np.array_equal(contour, I_CONTOURS):
        return "I"
    else:
        return "None"

In [7]:
pydirectinput.PAUSE = 0.001

contours = np.array([0, 0], dtype="int32")
firstTime = True
wasteT = True

time.sleep(1)
pydirectinput.click(x=540, y=540)
pydirectinput.keyDown("2")
time.sleep(0.3)
pydirectinput.keyUp("2")
time.sleep(1)
with mss() as sct:
    while True:

        screenShot = sct.grab(bounding_box)

        img = Image.frombytes(
            'RGB', 
            (screenShot.width, screenShot.height), 
            screenShot.rgb, 
        )

        img = np.array(img)[:, :, ::-1].copy()
        
        img = cv2.putText(img, contourToMino(contours), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        if not firstTime:
            if contourToMino(contours) == "T":
                if wasteT:
                    pydirectinput.press("shiftleft")
                    pydirectinput.press("left", presses=3)
                    pydirectinput.press("s")
                    wasteT = False
                else:
                    pydirectinput.press("d")
                    pydirectinput.press("left", presses=1)
                    pydirectinput.press('w')
                    pydirectinput.press("d")
                    pydirectinput.press("s")
            elif contourToMino(contours) == "J":
                pydirectinput.press("d")
                pydirectinput.press("left", presses=4)
                pydirectinput.press("s")
            elif contourToMino(contours) == "L":
                pydirectinput.press("a")
                pydirectinput.press("right", presses=4)
                pydirectinput.press("s")
            elif contourToMino(contours) == "S":
                pydirectinput.press("right", presses=1)
                pydirectinput.press("s")
            elif contourToMino(contours) == "O":
                pydirectinput.press("right", presses=1)
                pydirectinput.press('w')
                pydirectinput.press("right", presses=1)
                pydirectinput.press("s")
            elif contourToMino(contours) == "Z":
                pydirectinput.press("d")
                pydirectinput.press("left", presses=2)
                pydirectinput.press('w')
                pydirectinput.press("left", presses=1)
                pydirectinput.press("s")
            elif contourToMino(contours) == "I":
                pydirectinput.press("d")
                pydirectinput.press("right", presses=4)
                pydirectinput.press("s")
            #time.sleep(2)

        crop = img[57:115,185:265]

        #blur = cv2.GaussianBlur(img, (5, 5), 0)
        blur = cv2.bilateralFilter(crop,9,75,75)

        # grayscale
        gray = cv2.cvtColor(np.array(blur), cv2.COLOR_BGR2GRAY)

        #threshold
        thresh = cv2.threshold(gray, 64, 255, cv2.THRESH_BINARY)[1]

        # get contour bounding boxes and draw on copy of input
        contours = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours = contours[0] if len(contours) == 2 else contours[1]
        
        cv2.drawContours(crop, contours, -1, (255,255,255), 2)
        # view result
        img[57:115,185:265] = crop
        cv2.imshow('screen', np.array(img))

        if firstTime:
            firstTime = False
            pydirectinput.click()
            time.sleep(3)
            # cv2.destroyAllWindows()
            # plt.imshow(cv2.cvtColor(np.array(img), cv2.COLOR_BGR2RGB))
            # break

        if (cv2.waitKey(1) & 0xFF) == ord('q'):
        #if keyboard.is_pressed("q"):
            cv2.destroyAllWindows()
            #plt.imshow(cv2.cvtColor(np.array(img), cv2.COLOR_BGR2RGB))
            break

In [ ]:
#(715, 504) again button

In [1]:
cv2.imshow(" ", np.array(img))
cv2.waitKey(1)
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined